In [1]:
import numpy as np
from numba.typed import List as NumbaList
from BPnumba.NumAG import CalcFi,create_intidivual,Ind
from BPnumba.GeneticOperators import InstancePob,CreatePoblation,CreatePermutation,InverseMutation

from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from numba import njit,prange,objmode
from typing import List
import time
import random

In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]
contenedor =np.asanyarray( P2A2[0],dtype=np.int64)
datos =np.asanyarray(P2A2[1],dtype=np.int64)
idBoxes = np.arange(1,len(datos)+1,dtype=np.int64)
n=len(datos)

In [1017]:
lst = np.asanyarray([1,4,5,2,3,6,10,11,22],dtype=np.int64)
lst

array([ 1,  4,  5,  2,  3,  6, 10, 11, 22], dtype=int64)

In [1022]:
SwapSeqIndex(lst,4)

[ 6 10 11 22] [1 4 5 2]


array([ 6, 10, 11, 22,  3,  1,  4,  5,  2], dtype=int64)

In [1012]:
RRIS(lst,4)

array([22, 11, 10,  6,  3,  1,  4,  5,  2], dtype=int64)

In [861]:
InstancePob(CreatePoblation(10,idBoxes),datos,contenedor)
print('compiled InstancePob')

compiled InstancePob


In [ ]:
poblation = CreatePoblation(50,idBoxes)

In [ ]:
ColonyWorker = InstancePob(poblation,datos,contenedor)

In [ ]:
#Busqueda local de abejas para todas las trabajadoras y calcila Pi
fail = np.zeros(len(ColonyWorker))
for i in np.arange(len(ColonyWorker)):
    k = random.randint(0,len(datos)-1)
    j = random.randint(0,len(ColonyWorker)-1)
    while i == j:
        j = random.randint(0,len(ColonyWorker)-1)
    beeK = ColonyWorker[j]
    b2gen = ColonyWorker[i].genome.copy()
    b2gen[k] = ColonyWorker[i].genome[k] + round(random.uniform(-1,1)*(ColonyWorker[i].genome[k]-beeK.genome[k]))
    if b2gen[k] < 1:
        if ColonyWorker[i].genome[k] ==1:
            b2gen[k] = 2
        else:
            b2gen[k] = 1
    elif b2gen[k] >n:
        if ColonyWorker[i].genome[k] ==n:
            b2gen[k] = n-1
        else:
            b2gen[k] = n
    for l in np.arange(len(datos)):
        if l != k  and b2gen[l] == ColonyWorker[i].genome[k]:
            b2gen[l]=ColonyWorker[i].genome[k]
            break
    bee = create_intidivual(b2gen)
    CalcFi(bee,datos,contenedor)
    if bee.fi > ColonyWorker[i].fi:
        ColonyWorker[i]=bee
    else:
        fail[i]+=1


In [ ]:
fail